## Scraping a Bookstore web page:


## Scrapping the book webpage:

In [13]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [14]:
link='https://books.toscrape.com/catalogue/page-1.html'
res=requests.get(link)
soup=BeautifulSoup(res.text,'html.parser')

In [15]:
data=[]
for page in tqdm(range(1,51)):
    link=f'https://books.toscrape.com/catalogue/page-{page}.html'
    res=requests.get(link)
    soup=BeautifulSoup(res.text,'html.parser')

    for sp in soup.find_all('li',class_="col-xs-6 col-sm-4 col-md-3 col-lg-3"):
           
    
        link            ='https://books.toscrape.com/catalogue/'+str(sp.find_all('a')[-1].get('href'))
        title           =sp.find_all('a')[1].get('title')
        rating            =sp.find('p').get('class')[-1]
        price           =sp.find('p',class_='price_color').text[1:]
        availability    =sp.find('p', class_="instock availability").text.strip()
        
        
        data.append([title,link,rating,price,availability])

100%|██████████| 50/50 [01:39<00:00,  1.99s/it]


In [16]:
columns = ['Title', 'link_of_the_book', 'Rating', 'Price', 'Availability']
df=pd.DataFrame(data,columns=columns)
    

In [17]:
df.to_csv('books.csv',index=False)

### New scrapping :

In [6]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm


In [7]:
df=pd.read_csv("books.csv")

In [8]:
df.sample(3)

,Title,link_of_the_book,Rating,Price,Availability
747,The Guernsey Literary and Potato Peel Pie Society,https://books.toscrape.com/catalogue/the-guern...,One,£49.53,In stock
598,The False Prince (The Ascendance Trilogy #1),https://books.toscrape.com/catalogue/the-false...,Five,£56.00,In stock
261,Superman Vol. 1: Before Truth (Superman by Gen...,https://books.toscrape.com/catalogue/superman-...,Five,£11.89,In stock


In [39]:
data=[]
for link in tqdm(df['link_of_the_book']):
    res=requests.get(link)
    soup=BeautifulSoup(res.text,'html.parser')
    
    book_genre=soup.find('ul',class_='breadcrumb').find_all('a')[-1].text
    list=soup.find('div',class_="col-sm-6 product_main").find_all('p',class_='instock availability')[-1].text.strip()
    No_of_items_available=list.split(' ')[2].split('(')[1]
    upc_no=soup.find('table',class_='table table-striped').find('td').text

    data.append([book_genre,list,No_of_items_available,upc_no])
    

100%|██████████| 1000/1000 [27:46<00:00,  1.67s/it]


In [41]:
df_1=pd.DataFrame(data,columns=['Genre','unnamed_0','No_of_items_available','Book_ID'])

In [43]:
df_1.to_csv('Books_details.csv',index=False)

In [45]:
df_1=pd.read_csv('Books_details.csv')

## Merge the two different csv file:


In [44]:
df.head(2)

,Title,link_of_the_book,Rating,Price,Availability
0,A Light in the Attic,https://books.toscrape.com/catalogue/a-light-i...,Three,£51.77,In stock
1,Tipping the Velvet,https://books.toscrape.com/catalogue/tipping-t...,One,£53.74,In stock


In [46]:
df_1.head(2)

,Genre,unnamed_0,No_of_items_available,Book_ID
0,Poetry,In stock (22 available),22,a897fe39b1053632
1,Historical Fiction,In stock (20 available),20,90fa61229261140a


In [53]:
df_new=pd.DataFrame()

In [55]:
df_new['Genre']=df_1['Genre']
df_new['Title']=df['Title']
df_new['Price']=df['Price']
df_new['Rating']=df['Rating']
df_new['Availability']=df['Availability']
df_new['No_of_items_available']=df_1['No_of_items_available']
df_new['Book_ID']=df_1['Book_ID']



In [57]:
df_new.sample(2)

,Genre,Title,Price,Rating,Availability,No_of_items_available,Book_ID
796,Nonfiction,Outliers: The Story of Success,£14.16,One,In stock,3,09a7b0b566ad6225
297,Fiction,I Am Pilgrim (Pilgrim #1),£10.60,Four,In stock,14,aeff7b87a8e83ebb


In [58]:
df_new.to_csv('New_Books.csv',index=False)